In [23]:
import org.apache.spark.ml.classification.BinaryLogisticRegressionSummary
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.functions._

In [24]:
import org.apache.spark.ml.classification

In [25]:
val rawData = sc.textFile("/home/jovyan/data/stumble_upon/train.tsv")

rawData = /home/jovyan/data/stumble_upon/train.tsv MapPartitionsRDD[77] at textFile at <console>:40


/home/jovyan/data/stumble_upon/train.tsv MapPartitionsRDD[77] at textFile at <console>:40

In [26]:
val header = rawData.first()
val rawData1 = rawData.filter(line => line != header)
val records = rawData1.map(line => line.split("\t"))

header = "url"	"urlid"	"boilerplate"	"alchemy_category"	"alchemy_category_score"	"avglinksize"	"commonlinkratio_1"	"commonlinkratio_2"	"commonlinkratio_3"	"commonlinkratio_4"	"compression_ratio"	"embed_ratio"	"framebased"	"frameTagRatio"	"hasDomainLink"	"html_ratio"	"image_ratio"	"is_news"	"lengthyLinkDomain"	"linkwordscore"	"news_front_page"	"non_markup_alphanum_characters"	"numberOfLinks"	"numwords_in_url"	"parametrizedLinkRatio"	"spelling_errors_ratio"	"label"
rawData1 = MapPartitionsRDD[78] at filter at <console>:48
records = MapPartitionsRDD[79] at map at <console>:49


MapPartitionsRDD[79] at map at <console>:49

In [27]:
case class StumbleUpon(numberOfLinks: Double, numwords_in_url: Double, parametrizedLinkRatio: Double,
                       spelling_errors_ratio: Double, label: Int)
val data = records.map{ r =>
    val trimmed = r.map(_.replaceAll("\"", ""))
    val label = trimmed(r.size - 1).toInt
    val features = trimmed.slice(22, r.size - 1).map(d => if (d == "?") 0.0 else d.toDouble)
    assert(features.size == 4)
    StumbleUpon(features(0), features(1), features(2), features(3), label)
}

defined class StumbleUpon
data = MapPartitionsRDD[80] at map at <console>:25


MapPartitionsRDD[80] at map at <console>:25

In [28]:
data.first()

StumbleUpon(170.0,8.0,0.152941176,0.079129575,0)

In [29]:
val dataDF = data.toDF()

dataDF = [numberOfLinks: double, numwords_in_url: double ... 3 more fields]


[numberOfLinks: double, numwords_in_url: double ... 3 more fields]

In [30]:
dataDF.show(3)

+-------------+---------------+---------------------+---------------------+-----+
|numberOfLinks|numwords_in_url|parametrizedLinkRatio|spelling_errors_ratio|label|
+-------------+---------------+---------------------+---------------------+-----+
|        170.0|            8.0|          0.152941176|          0.079129575|    0|
|        187.0|            9.0|          0.181818182|          0.125448029|    1|
|        258.0|           11.0|          0.166666667|          0.057613169|    1|
+-------------+---------------+---------------------+---------------------+-----+
only showing top 3 rows



In [31]:
val featureCols = Array("numberOfLinks", "numwords_in_url", "parametrizedLinkRatio", "spelling_errors_ratio")
val assembler = new VectorAssembler().setInputCols(featureCols).setOutputCol("features")
val df2 = assembler.transform(dataDF)

featureCols = Array(numberOfLinks, numwords_in_url, parametrizedLinkRatio, spelling_errors_ratio)
assembler = vecAssembler_6d3fd382cc34
df2 = [numberOfLinks: double, numwords_in_url: double ... 4 more fields]


[numberOfLinks: double, numwords_in_url: double ... 4 more fields]

In [32]:
df2.show(3)

+-------------+---------------+---------------------+---------------------+-----+--------------------+
|numberOfLinks|numwords_in_url|parametrizedLinkRatio|spelling_errors_ratio|label|            features|
+-------------+---------------+---------------------+---------------------+-----+--------------------+
|        170.0|            8.0|          0.152941176|          0.079129575|    0|[170.0,8.0,0.1529...|
|        187.0|            9.0|          0.181818182|          0.125448029|    1|[187.0,9.0,0.1818...|
|        258.0|           11.0|          0.166666667|          0.057613169|    1|[258.0,11.0,0.166...|
+-------------+---------------+---------------------+---------------------+-----+--------------------+
only showing top 3 rows



In [33]:
val splitSeed = 5043
val Array(trainingData, testData) = df2.randomSplit(Array(0.7, 0.3), splitSeed)

splitSeed = 5043
trainingData = [numberOfLinks: double, numwords_in_url: double ... 4 more fields]
testData = [numberOfLinks: double, numwords_in_url: double ... 4 more fields]


[numberOfLinks: double, numwords_in_url: double ... 4 more fields]

In [34]:
val lr = new LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)
val model = lr.fit(trainingData)

lr = logreg_dbc1f91d4603
model = logreg_dbc1f91d4603


logreg_dbc1f91d4603

In [35]:
val predictions = model.transform(testData)

predictions = [numberOfLinks: double, numwords_in_url: double ... 7 more fields]


[numberOfLinks: double, numwords_in_url: double ... 7 more fields]

In [36]:
predictions.show(3)

+-------------+---------------+---------------------+---------------------+-----+--------------------+--------------------+--------------------+----------+
|numberOfLinks|numwords_in_url|parametrizedLinkRatio|spelling_errors_ratio|label|            features|       rawPrediction|         probability|prediction|
+-------------+---------------+---------------------+---------------------+-----+--------------------+--------------------+--------------------+----------+
|          1.0|            2.0|                  0.0|          0.063270991|    1|[1.0,2.0,0.0,0.06...|[-0.0370059391125...|[0.49074957085638...|       1.0|
|          2.0|            0.0|                  0.0|          0.105263158|    0|[2.0,0.0,0.0,0.10...|[-0.0370059391125...|[0.49074957085638...|       1.0|
|          2.0|            1.0|                  0.0|          0.089552239|    0|[2.0,1.0,0.0,0.08...|[-0.0370059391125...|[0.49074957085638...|       1.0|
+-------------+---------------+---------------------+-----------

In [37]:
val trainingSummary = model.summary
val objectiveHistory = trainingSummary.objectiveHistory
objectiveHistory.foreach(loss => println(loss))

0.6929760299168406


trainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@7c65e2f8
objectiveHistory = Array(0.6929760299168406)


[0.6929760299168406]

In [38]:
val binarySummary = trainingSummary.asInstanceOf[BinaryLogisticRegressionSummary]

binarySummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@7c65e2f8


org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@7c65e2f8

In [39]:
val roc = binarySummary.roc
roc.show()
println(binarySummary.areaUnderROC)

+---+---+
|FPR|TPR|
+---+---+
|0.0|0.0|
|1.0|1.0|
|1.0|1.0|
+---+---+

0.5


roc = [FPR: double, TPR: double]


[FPR: double, TPR: double]

In [40]:
// Set the model threshold to maximize F-Measure
val fMeasure = binarySummary.fMeasureByThreshold
val fm = fMeasure.col("F-Measure")
val maxFMeasure = fMeasure.select(max("F-Measure")).head().getDouble(0)
val bestThreshold = fMeasure.where($"F-Measure" === maxFMeasure).select("threshold").head().getDouble(0)
model.setThreshold(bestThreshold)

fMeasure = [threshold: double, F-Measure: double]
fm = F-Measure
maxFMeasure = 0.674838872741059
bestThreshold = 0.5092504291436201


logreg_dbc1f91d4603

In [41]:
val evaluator = new BinaryClassificationEvaluator().setLabelCol("label")

evaluator = binEval_033132bf8cec


binEval_033132bf8cec

In [42]:
val accuracy = evaluator.evaluate(predictions)

accuracy = 0.5


0.5